1 下载OSM路网

In [1]:
import osmnx as ox
import time
from shapely.geometry import Polygon
import os
import numpy as np

def save_graph_shapefile_directional(G, filepath=None, encoding="utf-8"):
    # default filepath if none was provided
    if filepath is None:
        filepath = os.path.join(ox.settings.data_folder, "graph_shapefile")

    # if save folder does not already exist, create it (shapefiles
    # get saved as set of files)
    if not filepath == "" and not os.path.exists(filepath):
        os.makedirs(filepath)
    filepath_nodes = os.path.join(filepath, "nodes.shp")
    filepath_edges = os.path.join(filepath, "edges.shp")

    # convert undirected graph to gdfs and stringify non-numeric columns
    gdf_nodes, gdf_edges = ox.utils_graph.graph_to_gdfs(G)
    gdf_nodes = ox.io._stringify_nonnumeric_cols(gdf_nodes)#函数将非数值列的数据转换为字符串，确保可以正确保存为 shapefile
    gdf_edges = ox.io._stringify_nonnumeric_cols(gdf_edges)
    # We need an unique ID for each edge
    gdf_edges["fid"] = np.arange(0, gdf_edges.shape[0], dtype='int')
    # save the nodes and edges as separate ESRI shapefiles
    gdf_nodes.to_file(filepath_nodes, encoding=encoding)
    gdf_edges.to_file(filepath_edges, encoding=encoding)

print("osmnx version",ox.__version__)
'''
# Download by a bounding box
bounds = (17.4110711999999985,18.4494298999999984,59.1412578999999994,59.8280297000000019)
x1,x2,y1,y2 = bounds
boundary_polygon = Polygon([(x1,y1),(x2,y1),(x2,y2),(x1,y2)])
G = ox.graph_from_polygon(boundary_polygon, network_type='drive')
start_time = time.time()
save_graph_shapefile_directional(G, filepath='./network-new')
print("--- %s seconds ---" % (time.time() - start_time))
'''
# Download by place name
place_name = "wuhan, china"
G= ox.graph_from_place(place_name, network_type='drive')
save_graph_shapefile_directional(G, filepath='wuhan')
'''
# Download by a boundary polygon in geojson
import osmnx as ox
from shapely.geometry import shape
json_file = open("stockholm_boundary.geojson")
import json
data = json.load(json_file)
boundary_polygon = shape(data["features"][0]['geometry'])
G = ox.graph_from_polygon(boundary_polygon, network_type='drive')
save_graph_shapefile_directional(G, filepath='stockholm')

'''

osmnx version 1.8.0


ConnectTimeout: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?format=json&polygon_geojson=1&dedupe=0&limit=50&q=wuhan%2C+china (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fb625fd6250>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=180)'))

2 更新id

In [4]:
import geopandas as gpd
import pandas as pd
import numpy as np


gdf=gpd.read_file('./wuhan/multiID.shp') 
gdf.sort_values(by='osmid',inplace=True)
gdf['idnew'] = -1

# Create a dictionary to store the first index for each unique 'osmid'
osmid_first_index = {}

# Iterate through the DataFrame and assign values to 'idnew'
for index, row in gdf.iterrows():
    #该行osmid的值。
    osmid = row['osmid']
    #如果osmid在字典中，该位置的idnew行等于osmid对应的
    if osmid in osmid_first_index:
        gdf.at[index, 'idnew'] = osmid_first_index[osmid]
    #不在就先存进去，再赋值
    else:
        osmid_first_index[osmid] = index
        gdf.at[index, 'idnew'] = index


gdf_single=gpd.read_file('./wuhan/singleID.shp') 
gdf_single['idnew']=gdf_single['osmid']

mergedf=pd.concat([gdf,gdf_single],axis=0)
mergedf.to_file('OSMdowload.shp', encoding='utf-8')

3 处理路网

In [3]:
import geopandas as gpd
import pandas as pd
import numpy as np


dir='D:/Amap/processdata/路网与边界/wuhan/'
oirRoad=gpd.read_file(dir+'no_dissovle_dropdup/osm_gcj_14160_dropdup_limit.shp')
dissolveRoad=gpd.read_file(dir+'osm_gcj_5659_xy12/dissovle_osm_gcj_5659_limitspeed.shp')


morethan2000List=dissolveRoad[dissolveRoad['length']>2000]['idnew'].tolist()


In [8]:
oirRoad.loc[oirRoad['idnew'].isin(morethan2000List)].to_file(dir+'osm_gcj_5659_xy12/selFromNodissovle_osm5659L2000.shp',driver='ESRI Shapefile',encoding='utf-8')